## Set Up / Imports

Main items of importance here are GPT-2, markovify, and a few other python libraries for later use.

In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
!pip install markovify
!pip install dominate

In [ ]:
# GPT-2 Stuff

import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

# Added Stuff

import random
import markovify
import dominate
from dominate.tags import *

In [ ]:
# Class(es)
# Don't necessarily need these, but here's to practice

class character(object):
    def __init__(self,name,age,race,title,profile,history,skills,opinions):
        self.name = name
        self.age = age
        self.race = race
        self.title = title
        self.profile = profile
        self.history = history
        self.skills = skills
        self.opinions = opinions

## Creating Initial Character Profile

Build initial character profile off of some more basic generation.

I.E. Random selection, but somewhat based off tracery. Might actually convert to tracery at some point.

In [ ]:
# Generate name and title
def generateName():
    with open("sources/btn_givennames.txt",encoding="UTF-8") as namebank:
        text = namebank.read()

        # remove /ignore gender markings
        # and pick a name at random
        text = text.replace("	m","").replace("	f","")
        names = text.split()
        name = random.choice(names)

        return name

# Generate an age from 25 - 60
def generateAge():
    age = random.randint(25,100)
    return age

# Give the character a title like The Stone-Crusher or something
def generateTitle():
  colors = ["red","blue","green","yellow","pink","purple",
            "orange","black","white","brown","grey","silver","gold"]
  jobs = ["necromancer","paladin","priest","warlock","ranger","bard","warden",
             "thief","scout","cleric","templar","gunslinger","alchemist"]
  things = ["stone","grass","water","mist","music","life",
            "blood", "land","kingdom","mountain",]
  descript = ["crusher","cutter","waster","healer","saviour","fool",
              "taker","enigma"]

  color = random.choice(colors).title()
  job = random.choice(jobs).title()
  thing = random.choice(things).title()
  doer = random.choice(descript).title()

# Less chance of thing-ish title because they've less options
  randint = random.randint(0,2)
  if randint == 0:
    title = "the "+color+" "+job+""
  if randint == 1:
    title = "the "+color+" "+job+""
  elif randint == 2:
    title = "the "+thing+" "+doer+""
  return title

def generateRace():
  races = ["human","elf","dwarf","orc","goblin",
           "beast man", "fae", "gnome", "lizard",
           "daemon", "elemental"]
           
  race = random.choice(races)
  return race  

# Have to call this now to use with gpt-2 prefix
name = generateName()
age = generateAge()
title = generateTitle()
race = generateRace()

chara = character(name,age,race,title,"","","","")

# Defining the Character with GPT-2
## Generating Profile & History

Taking the information created above, use GPT-2 to generate a short profile for the character. While I wanted to use a fine-tuned model, I ran into too many issues with RAM and having to re-boot page to use it effectively for the assignment.

In [ ]:
# Generate Profile
def generateProfile():
    profile = gpt2.generate(sess,
              model_name=model_name,
              prefix="Profile:\n\nName: "+chara.name+"\nRace: "+chara.race.title()+
                     "\nClass: "+chara.title.title()+"\nAge: "+str(chara.age)+"\nHobbies:",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=1,
              batch_size=1,
              return_as_list=True
    )
    return profile

# Taking elements from prior cell,
# compile profile with GPT-2 generation

convertProfileToString = ''.join(generateProfile())
chara.profile = convertProfileToString

f = open("resume_"+chara.name+".txt", "w")
f.write(chara.profile)
f.close()

In [ ]:
# Generating Work History
# Goal is to give each character like... 4-5 past jobs
def generateHistory(count):
    companies  = ["Fantasy Costo","The Bureau of Balance","The Prancing Pony","The Royal Court",
              "The Silver Hand", "The Court of Howls","Coins-A-Plenty","Pinch-A-Penance"]
    positions = ["Guest Attendant","Barkeep","Minstrel","Stablehand",
                "Innkeeper","Beggar","Servant","Butcher","Farmer","Ranchhand"]
    months = ["January","February","March","April","May","June","July",
              "August","September","October","November","December"]
    
    company = random.choice(companies)
    position = random.choice(positions)

    month = random.choice(months)
    month2 = random.choice(months)
    year = random.randint(1000,1100)
    year2 = year + (random.randint(1,5))

    run = count

    if run == 0:
      prefix = "Employment History: "+position+"\n"+company+"\n"+month+" "+str(year)+" - Present\n\nResponsibilities:"
    else:
      prefix = ""+position+"\n"+company+"\n"+month2+" "+str(year2)+" - "+month+" "+str(year)+"\n\nResponsibilities:"

    history = gpt2.generate(sess,
              model_name=model_name,
              prefix=prefix,
              length=50,
              temperature=0.7,
              top_p=0.9,
              nsamples=1,
              batch_size=1,
              return_as_list=True
    )
    return history

# Taking elements from prior cell,
# compile profile with GPT-2 generation
count = 0
while count <= 5:
  convertHistoryToString = ''.join(generateHistory(count))
  chara.history += convertHistoryToString
  chara.history += "\n\n"
  count += 1

f = open("resume_"+chara.name+".txt", "a")
f.write("\n\n"+chara.history)
f.close()

## Conducting an Interview

Using some of the previous information, mix Markovify and GPT-2 in order to create a pseudo character interview. Unfortunately, 5 questions seems to be the limit for GPT-2 before google colab crashes (whoops).

In [ ]:
# Set up Markovify's Model

with open("sources/interview.txt",encoding="UTF-8") as source:
    text = source.read()

model = markovify.Text(text)

questions = []
interview = []
opinions = ""
counter = 0

# Generate 5 questions from source text
# Can technically change number here to make itnerview bigger or smaller
for i in range(5):
  questions.append(model.make_sentence())

# Take questiions & add specific format to kind of push GPT-2's gen
for question in questions:
  question = str("Q: When asked '" + question + "' "+chara.name+" said\n\nA:")
  interview.append(question)

def generateOpinion(counter):
    count = counter
    opinion = gpt2.generate(sess,
                model_name=model_name,
                prefix=interview[count],
                length=100,
                temperature=0.7,
                top_p=0.9,
                nsamples=1,
                batch_size=1,
                return_as_list=True
    )
    return opinion

# Conduct interview
#while counter < 3:
while counter < len(interview):
  opinions += ''.join(generateOpinion(counter))
  opinions += "--\n\n"
  counter += 1

chara.opinions = opinions

f = open("resume_"+chara.name+".txt", "a")
f.write("\n\n"+chara.opinions)
f.close()

# SOURCES

## Namebank:
- Local: btn_givennames.txt
- From: https://www.behindthename.com/api/

## Question-bank:
- Character Interview
  - Local: interview.txt
  - From: https://thewritingkylie.com/blog/get-to-know-your-characters-with-character-interviews-115-questions
  - Note: Edited out a few section headers


# GPT-2 LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [ ]:
"""
This still doesn't produce what I want, so, begone

# Generate Skills
def generateSkills():
  skills = ["Shearing","Herding","Cooking","Leadership","Communication",
            "Problem-Solving","Puzzle-Solving","Pereverance","Ability to work under pressure",
            "Arcane Talent", "Storytelling", "Public Speaking", "Not Dying"]
  skill = random.choice(skills)

  skill_des = gpt2.generate(sess,
              model_name=model_name,
              prefix=""+skill.title()+"- History of",
              length=50,
              temperature=0.7,
              top_p=0.9,
              nsamples=1,
              batch_size=1,
              return_as_list=True
    )
  return skill_des

count = 0
while count <= 3:
  convertSkillsToString = ''.join(generateSkills())
  chara.skills += convertSkillsToString
  chara.skills += "\n\n"
  count += 1

f = open("resume_"+chara.name+".txt", "a")
f.write("\n\n"+chara.skills)
f.close()
"""